In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, PolynomialFeatures, scale
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression, RFE
import datetime
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer

import warnings # supress warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('600K US Housing Properties.csv', low_memory=False)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 28 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   property_url        600000 non-null  object 
 1   property_id         600000 non-null  int64  
 2   address             600000 non-null  object 
 3   street_name         599869 non-null  object 
 4   apartment           14815 non-null   object 
 5   city                599999 non-null  object 
 6   state               599999 non-null  object 
 7   latitude            529122 non-null  float64
 8   longitude           529122 non-null  float64
 9   postcode            599970 non-null  object 
 10  price               600000 non-null  float64
 11  bedroom_number      443845 non-null  float64
 12  bathroom_number     471733 non-null  float64
 13  price_per_unit      435365 non-null  float64
 14  living_space        447847 non-null  float64
 15  land_space          515119 non-nul

In [4]:
description = df.describe(include='all')
description

,property_url,property_id,address,street_name,apartment,city,state,latitude,longitude,postcode,...,property_type,property_status,year_build,total_num_units,listing_age,RunDate,agency_name,agent_name,agent_phone,is_owned_by_zillow
count,600000,6.000000e+05,600000,599869,14815,599999,599999,529122.000000,529122.000000,599970,...,600000,600000,0.0,0.0,600000.0,600000,444524,0.0,0.0,600000.000000
unique,600000,NaN,598588,339224,2664,7977,25,NaN,NaN,10820,...,7,2,NaN,NaN,NaN,1,34372,NaN,NaN,NaN
top,https://www.zillow.com/homedetails/3-Plat-83-1...,NaN,"(undisclosed Address), Rockport, TX 78382",(undisclosed Address),1,Chicago,TX,NaN,NaN,84043,...,SINGLE_FAMILY,FOR_SALE,NaN,NaN,NaN,2022-04-24 07:34:15,Coldwell Banker Realty,NaN,NaN,NaN
freq,1,NaN,36,1713,312,14138,146636,NaN,NaN,1102,...,354366,383365,NaN,NaN,NaN,600000,5936,NaN,NaN,NaN
mean,NaN,8.885042e+08,NaN,NaN,NaN,NaN,NaN,36.282379,-105.813906,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000498
std,NaN,9.724708e+08,NaN,NaN,NaN,NaN,NaN,5.673355,13.464633,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.022318
min,NaN,2.700000e+01,NaN,NaN,NaN,NaN,NaN,18.985142,-165.408250,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000
25%,NaN,5.402143e+07,NaN,NaN,NaN,NaN,NaN,32.612112,-117.346079,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000
50%,NaN,2.066090e+08,NaN,NaN,NaN,NaN,NaN,35.403568,-101.897378,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000
75%,NaN,2.066867e+09,NaN,NaN,NaN,NaN,NaN,39.661674,-95.354245,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000


In [5]:
df.columns

Index(['property_url', 'property_id', 'address', 'street_name', 'apartment',
       'city', 'state', 'latitude', 'longitude', 'postcode', 'price',
       'bedroom_number', 'bathroom_number', 'price_per_unit', 'living_space',
       'land_space', 'land_space_unit', 'broker_id', 'property_type',
       'property_status', 'year_build', 'total_num_units', 'listing_age',
       'RunDate', 'agency_name', 'agent_name', 'agent_phone',
       'is_owned_by_zillow'],
      dtype='object')

In [6]:
#Dropped the LOTs because this is another category and data from this category doesn't have many metrics that we use to predict
df = df.drop(df[df.property_type == 'LOT'].index)
df = df.drop(df[df.living_space == 0].index)
df = df.drop(df[df.price == 0].index)
df['longitude'] = -df['longitude']
#Selected just the Texas data to reduce column number so model can fit and predict
df = df[df.state == 'TX']

In [7]:
#Land space is an important metric for the price of the house but this dataset has multiple units
#All the values with 'acres' unit changed to 'sqft'
df.loc[df['land_space_unit']=='acres', 'land_space'] = df['land_space']*43560.00
df.loc[df['land_space_unit']=='acres', 'land_space_unit']='sqft'
df[['land_space','land_space_unit']]

,land_space,land_space_unit
180271,6969.600,sqft
180273,12632.400,sqft
180275,11325.600,sqft
180277,15246.000,sqft
180279,6000.000,sqft
...,...,...
446346,8145.720,sqft
446348,3920.400,sqft
446350,4356.000,sqft
446352,4299.372,sqft


In [8]:
df = df.drop(columns=['property_url', 'property_id', 'address', 'street_name', 'apartment', 'city', 'state', 'price_per_unit', 'land_space_unit', 'broker_id','property_status', 'year_build', 'total_num_units', 'listing_age',
       'RunDate', 'agency_name', 'agent_name', 'agent_phone',
       'is_owned_by_zillow'])

In [9]:
df_isna=df.isna()
df_isna

,latitude,longitude,postcode,price,bedroom_number,bathroom_number,living_space,land_space,property_type
180271,False,False,False,False,False,False,False,False,False
180273,False,False,False,False,False,False,False,False,False
180275,False,False,False,False,False,False,False,False,False
180277,False,False,False,False,False,False,False,False,False
180279,False,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...
446346,False,False,False,False,False,False,False,False,False
446348,False,False,False,False,False,False,False,False,False
446350,False,False,False,False,False,False,False,False,False
446352,False,False,False,False,False,False,False,False,False


In [10]:
df_isna.columns=[c for c in df_isna.columns]
df_isna

,latitude,longitude,postcode,price,bedroom_number,bathroom_number,living_space,land_space,property_type
180271,False,False,False,False,False,False,False,False,False
180273,False,False,False,False,False,False,False,False,False
180275,False,False,False,False,False,False,False,False,False
180277,False,False,False,False,False,False,False,False,False
180279,False,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...
446346,False,False,False,False,False,False,False,False,False
446348,False,False,False,False,False,False,False,False,False
446350,False,False,False,False,False,False,False,False,False
446352,False,False,False,False,False,False,False,False,False


In [11]:
df_isna.columns=[c+'_isna' for c in df_isna.columns]
df_isna

,latitude_isna,longitude_isna,postcode_isna,price_isna,bedroom_number_isna,bathroom_number_isna,living_space_isna,land_space_isna,property_type_isna
180271,False,False,False,False,False,False,False,False,False
180273,False,False,False,False,False,False,False,False,False
180275,False,False,False,False,False,False,False,False,False
180277,False,False,False,False,False,False,False,False,False
180279,False,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...
446346,False,False,False,False,False,False,False,False,False
446348,False,False,False,False,False,False,False,False,False
446350,False,False,False,False,False,False,False,False,False
446352,False,False,False,False,False,False,False,False,False


In [12]:
df=pd.concat([df.fillna(0), df_isna.astype(float)], axis=1)
df

,latitude,longitude,postcode,price,bedroom_number,bathroom_number,living_space,land_space,property_type,latitude_isna,longitude_isna,postcode_isna,price_isna,bedroom_number_isna,bathroom_number_isna,living_space_isna,land_space_isna,property_type_isna
180271,31.786737,106.428020,79903,239500.0,5.0,3.0,1692.0,6969.600,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180273,31.784021,106.404450,79925,165000.0,4.0,2.0,1650.0,12632.400,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180275,31.761911,106.432330,79905,118000.0,4.0,1.0,1918.0,11325.600,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180277,31.790165,106.435960,79903,414700.0,4.0,3.0,3119.0,15246.000,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180279,31.775480,106.438230,79905,260000.0,0.0,0.0,3267.0,6000.000,MULTI_FAMILY,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446346,33.059647,96.710945,75023,393000.0,3.0,2.0,1717.0,8145.720,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446348,33.044098,96.707720,75023,315000.0,3.0,3.0,1754.0,3920.400,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446350,33.069946,96.712920,75023,360000.0,4.0,3.0,2068.0,4356.000,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446352,33.070004,96.713900,75023,447000.0,4.0,3.0,2111.0,4299.372,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
description = df.describe(include='all')
description

,latitude,longitude,postcode,price,bedroom_number,bathroom_number,living_space,land_space,property_type,latitude_isna,longitude_isna,postcode_isna,price_isna,bedroom_number_isna,bathroom_number_isna,living_space_isna,land_space_isna,property_type_isna
count,113297.000000,113297.000000,113297,1.132970e+05,113297.000000,113297.000000,1.132970e+05,1.132970e+05,113297,113297.000000,113297.000000,113297.0,113297.0,113297.000000,113297.000000,113297.000000,113297.000000,113297.0
unique,NaN,NaN,1661,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,75126,NaN,NaN,NaN,NaN,NaN,SINGLE_FAMILY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,964,NaN,NaN,NaN,NaN,NaN,101682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,27.824578,87.700390,NaN,5.191826e+05,3.364184,2.664432,2.433314e+03,2.348514e+05,NaN,0.096905,0.096905,0.0,0.0,0.022048,0.005481,0.020724,0.194383,0.0
std,9.261089,28.787718,NaN,7.836710e+05,1.131400,1.208102,3.708981e+04,8.640966e+06,NaN,0.295829,0.295829,0.0,0.0,0.146841,0.073832,0.142460,0.395727,0.0
min,0.000000,0.000000,NaN,1.000000e+00,0.000000,0.000000,0.000000e+00,-1.089000e+04,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
25%,29.525587,95.413230,NaN,2.728000e+05,3.000000,2.000000,1.557000e+03,3.920400e+03,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
50%,30.108833,96.749040,NaN,3.800000e+05,3.000000,3.000000,2.035000e+03,7.270164e+03,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
75%,32.416286,97.836630,NaN,5.499000e+05,4.000000,3.000000,2.678000e+03,1.289376e+04,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [14]:
df.isna().sum()

latitude                0
longitude               0
postcode                0
price                   0
bedroom_number          0
bathroom_number         0
living_space            0
land_space              0
property_type           0
latitude_isna           0
longitude_isna          0
postcode_isna           0
price_isna              0
bedroom_number_isna     0
bathroom_number_isna    0
living_space_isna       0
land_space_isna         0
property_type_isna      0
dtype: int64

In [15]:
df.head()

,latitude,longitude,postcode,price,bedroom_number,bathroom_number,living_space,land_space,property_type,latitude_isna,longitude_isna,postcode_isna,price_isna,bedroom_number_isna,bathroom_number_isna,living_space_isna,land_space_isna,property_type_isna
180271,31.786737,106.42802,79903,239500.0,5.0,3.0,1692.0,6969.6,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180273,31.784021,106.40445,79925,165000.0,4.0,2.0,1650.0,12632.4,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180275,31.761911,106.43233,79905,118000.0,4.0,1.0,1918.0,11325.6,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180277,31.790165,106.43596,79903,414700.0,4.0,3.0,3119.0,15246.0,SINGLE_FAMILY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180279,31.775480,106.43823,79905,260000.0,0.0,0.0,3267.0,6000.0,MULTI_FAMILY,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [16]:
# A crucial categorical feature 'property_type' is categorical variable so I wanted to transform that to a numeric one
dummies=pd.get_dummies(df[['postcode','property_type']], drop_first=True)
dummies

,postcode_11111,postcode_73870,postcode_75001,postcode_75002,postcode_75006,postcode_75007,postcode_75009,postcode_75010,postcode_75013,postcode_75017,...,postcode_79934,postcode_79935,postcode_79936,postcode_79938,postcode_97316,property_type_CONDO,property_type_MANUFACTURED,property_type_MULTI_FAMILY,property_type_SINGLE_FAMILY,property_type_TOWNHOUSE
180271,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
180273,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
180275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
180277,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
180279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
446348,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
446350,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
446352,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
df = pd.concat([df,dummies],axis='columns')
df

,latitude,longitude,postcode,price,bedroom_number,bathroom_number,living_space,land_space,property_type,latitude_isna,...,postcode_79934,postcode_79935,postcode_79936,postcode_79938,postcode_97316,property_type_CONDO,property_type_MANUFACTURED,property_type_MULTI_FAMILY,property_type_SINGLE_FAMILY,property_type_TOWNHOUSE
180271,31.786737,106.428020,79903,239500.0,5.0,3.0,1692.0,6969.600,SINGLE_FAMILY,0.0,...,0,0,0,0,0,0,0,0,1,0
180273,31.784021,106.404450,79925,165000.0,4.0,2.0,1650.0,12632.400,SINGLE_FAMILY,0.0,...,0,0,0,0,0,0,0,0,1,0
180275,31.761911,106.432330,79905,118000.0,4.0,1.0,1918.0,11325.600,SINGLE_FAMILY,0.0,...,0,0,0,0,0,0,0,0,1,0
180277,31.790165,106.435960,79903,414700.0,4.0,3.0,3119.0,15246.000,SINGLE_FAMILY,0.0,...,0,0,0,0,0,0,0,0,1,0
180279,31.775480,106.438230,79905,260000.0,0.0,0.0,3267.0,6000.000,MULTI_FAMILY,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446346,33.059647,96.710945,75023,393000.0,3.0,2.0,1717.0,8145.720,SINGLE_FAMILY,0.0,...,0,0,0,0,0,0,0,0,1,0
446348,33.044098,96.707720,75023,315000.0,3.0,3.0,1754.0,3920.400,SINGLE_FAMILY,0.0,...,0,0,0,0,0,0,0,0,1,0
446350,33.069946,96.712920,75023,360000.0,4.0,3.0,2068.0,4356.000,SINGLE_FAMILY,0.0,...,0,0,0,0,0,0,0,0,1,0
446352,33.070004,96.713900,75023,447000.0,4.0,3.0,2111.0,4299.372,SINGLE_FAMILY,0.0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
df = df.drop(columns=['postcode','property_type'])
df

,latitude,longitude,price,bedroom_number,bathroom_number,living_space,land_space,latitude_isna,longitude_isna,postcode_isna,...,postcode_79934,postcode_79935,postcode_79936,postcode_79938,postcode_97316,property_type_CONDO,property_type_MANUFACTURED,property_type_MULTI_FAMILY,property_type_SINGLE_FAMILY,property_type_TOWNHOUSE
180271,31.786737,106.428020,239500.0,5.0,3.0,1692.0,6969.600,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
180273,31.784021,106.404450,165000.0,4.0,2.0,1650.0,12632.400,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
180275,31.761911,106.432330,118000.0,4.0,1.0,1918.0,11325.600,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
180277,31.790165,106.435960,414700.0,4.0,3.0,3119.0,15246.000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
180279,31.775480,106.438230,260000.0,0.0,0.0,3267.0,6000.000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446346,33.059647,96.710945,393000.0,3.0,2.0,1717.0,8145.720,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
446348,33.044098,96.707720,315000.0,3.0,3.0,1754.0,3920.400,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
446350,33.069946,96.712920,360000.0,4.0,3.0,2068.0,4356.000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
446352,33.070004,96.713900,447000.0,4.0,3.0,2111.0,4299.372,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
df.dtypes

latitude                       float64
longitude                      float64
price                          float64
bedroom_number                 float64
bathroom_number                float64
                                ...   
property_type_CONDO              uint8
property_type_MANUFACTURED       uint8
property_type_MULTI_FAMILY       uint8
property_type_SINGLE_FAMILY      uint8
property_type_TOWNHOUSE          uint8
Length: 1681, dtype: object

In [20]:
df["price_per_sqfoot"] = df["price"] / df["living_space"]

In [21]:
q_low = df["bedroom_number"].quantile(0.01)
q_hi  = df["bedroom_number"].quantile(0.99)

df = df[(df["bedroom_number"] < q_hi) & (df["bedroom_number"] > q_low)]

In [22]:
q_low = df["bathroom_number"].quantile(0.01)
q_hi  = df["bathroom_number"].quantile(0.99)

df = df[(df["bathroom_number"] < q_hi) & (df["bathroom_number"] > q_low)]

In [23]:
q_low = df["living_space"].quantile(0.01)
q_hi  = df["living_space"].quantile(0.99)

df = df[(df["living_space"] < q_hi) & (df["living_space"] > q_low)]

In [24]:
q_low = df["land_space"].quantile(0.01)
q_hi  = df["land_space"].quantile(0.99)

df = df[(df["land_space"] < q_hi) & (df["land_space"] > q_low)]

In [25]:
q_low = df["price"].quantile(0.01)
q_hi  = df["price"].quantile(0.99)

df = df[(df["price"] < q_hi) & (df["price"] > q_low)]

In [26]:
q_low = df["price_per_sqfoot"].quantile(0.01)
q_hi  = df["price_per_sqfoot"].quantile(0.99)

df = df[(df["price_per_sqfoot"] < q_hi) & (df["price_per_sqfoot"] > q_low)]

In [27]:
description = df.describe(include='all')
description

,latitude,longitude,price,bedroom_number,bathroom_number,living_space,land_space,latitude_isna,longitude_isna,postcode_isna,...,postcode_79935,postcode_79936,postcode_79938,postcode_97316,property_type_CONDO,property_type_MANUFACTURED,property_type_MULTI_FAMILY,property_type_SINGLE_FAMILY,property_type_TOWNHOUSE,price_per_sqfoot
count,72944.000000,72944.000000,7.294400e+04,72944.000000,72944.000000,72944.000000,7.294400e+04,72944.000000,72944.000000,72944.0,...,72944.000000,72944.000000,72944.000000,72944.000000,72944.000000,72944.000000,72944.000000,72944.000000,72944.000000,72944.000000
mean,27.634300,87.065568,4.355381e+05,3.498410,2.736024,2214.939611,2.996676e+04,0.103627,0.103627,0.0,...,0.000165,0.000781,0.002714,0.000014,0.017534,0.023840,0.005031,0.920528,0.032929,195.456814
std,9.534342,29.660041,2.286204e+05,0.700135,0.812543,744.852660,8.977662e+04,0.304779,0.304779,0.0,...,0.012825,0.027943,0.052030,0.003703,0.131251,0.152552,0.070753,0.270476,0.178453,69.131821
min,0.000000,0.000000,9.995000e+04,1.000000,1.750000,958.000000,3.850000e-02,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,77.599586
25%,29.525587,95.417113,2.800000e+05,3.000000,2.000000,1651.000000,6.011280e+03,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,149.355999
50%,30.104156,96.776285,3.790000e+05,3.000000,3.000000,2070.000000,7.884360e+03,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,182.727273
75%,32.464257,97.817865,5.250000e+05,4.000000,3.000000,2644.000000,1.268903e+04,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,224.919961
max,36.461475,106.631890,1.699999e+06,5.000000,5.500000,4834.000000,1.053716e+06,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,516.173434


In [28]:
features=df.describe().columns.drop(['price', 'price_per_sqfoot'])
features

Index(['latitude', 'longitude', 'bedroom_number', 'bathroom_number',
       'living_space', 'land_space', 'latitude_isna', 'longitude_isna',
       'postcode_isna', 'price_isna',
       ...
       'postcode_79934', 'postcode_79935', 'postcode_79936', 'postcode_79938',
       'postcode_97316', 'property_type_CONDO', 'property_type_MANUFACTURED',
       'property_type_MULTI_FAMILY', 'property_type_SINGLE_FAMILY',
       'property_type_TOWNHOUSE'],
      dtype='object', length=1680)

In [29]:
targets=['price','price_per_sqfoot']
targets

['price', 'price_per_sqfoot']

In [30]:
X=df[features]
X

,latitude,longitude,bedroom_number,bathroom_number,living_space,land_space,latitude_isna,longitude_isna,postcode_isna,price_isna,...,postcode_79934,postcode_79935,postcode_79936,postcode_79938,postcode_97316,property_type_CONDO,property_type_MANUFACTURED,property_type_MULTI_FAMILY,property_type_SINGLE_FAMILY,property_type_TOWNHOUSE
180271,31.786737,106.428020,5.0,3.0,1692.0,6969.600,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
180273,31.784021,106.404450,4.0,2.0,1650.0,12632.400,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
180277,31.790165,106.435960,4.0,3.0,3119.0,15246.000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
180281,31.785316,106.411150,4.0,2.0,1800.0,6098.400,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
180288,31.799797,106.437600,3.0,3.0,2790.0,6098.400,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446346,33.059647,96.710945,3.0,2.0,1717.0,8145.720,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
446348,33.044098,96.707720,3.0,3.0,1754.0,3920.400,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
446350,33.069946,96.712920,4.0,3.0,2068.0,4356.000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
446352,33.070004,96.713900,4.0,3.0,2111.0,4299.372,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [31]:
y=df[targets[0]]
y

180271    239500.0
180273    165000.0
180277    414700.0
180281    174950.0
180288    249000.0
            ...   
446346    393000.0
446348    315000.0
446350    360000.0
446352    447000.0
446354    377000.0
Name: price, Length: 72944, dtype: float64

In [32]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=42)
len(X_train),len(X_test),len(y_train),len(y_test)

(51060, 21884, 51060, 21884)

In [33]:
numeric=['latitude', 'longitude', 'bedroom_number', 'bathroom_number', 'living_space', 'land_space']
sc=StandardScaler()
X_train[numeric]=sc.fit_transform(X_train[numeric])
X_test[numeric]=sc.transform(X_test[numeric])

**Linear Regression Model**

In [34]:
model = LinearRegression()
model.fit(X_train , y_train)

LinearRegression()

In [35]:
y_pred = model.predict(X_test)

In [36]:
from sklearn import metrics
print('MAE: ',metrics.mean_absolute_error(y_test,y_pred))
print('MSE: ',metrics.mean_squared_error(y_test,y_pred))
print('RMSE: ',metrics.mean_squared_error(y_test,y_pred, squared=False))

MAE:  1643680124410.9292
MSE:  3.7145264197266924e+27
RMSE:  60946914767908.414


In [37]:
print('R square: ',model.score(X_train , y_train))

R square:  0.7539199746616704


**Ridge Regression Model**

In [38]:
clf = Ridge(alpha=0.1)
clf.fit(X_train , y_train)

Ridge(alpha=0.1)

In [39]:
clf_pred = clf.predict(X_test)

In [40]:
print('MAE: ',metrics.mean_absolute_error(y_test,clf_pred))
print('MSE: ',metrics.mean_squared_error(y_test,clf_pred))
print('RMSE: ',metrics.mean_squared_error(y_test,clf_pred, squared=False))

MAE:  80138.34283379253
MSE:  13958952705.600327
RMSE:  118148.01185631659


In [41]:
print('R square: ',clf.score(X_train , y_train))

R square:  0.7538958005248191


**Lasso Regression Model**

In [42]:
clf1 = Lasso(alpha=0.1)
clf1.fit(X_train , y_train)

Lasso(alpha=0.1)

In [43]:
clf1_pred = clf1.predict(X_test)

In [44]:
print('MAE: ',metrics.mean_absolute_error(y_test,clf1_pred))
print('MSE: ',metrics.mean_squared_error(y_test,clf1_pred))
print('RMSE: ',metrics.mean_squared_error(y_test,clf1_pred, squared=False))

MAE:  80148.62126761906
MSE:  13962546453.050636
RMSE:  118163.21954419927


In [45]:
print('R square: ',clf1.score(X_train , y_train))

R square:  0.7539081161073248


**Random Forest Model**

In [46]:
from sklearn.ensemble import RandomForestRegressor
rfm=RandomForestRegressor()
rfm.fit(X_train,y_train)

RandomForestRegressor()

In [47]:
print('R square: ',rfm.score(X_train,y_train))

R square:  0.9694820795282593


In [48]:
rfm_pred=rfm.predict(X_test)

In [49]:
print('MAE: ',metrics.mean_absolute_error(y_test,rfm_pred))
print('MSE: ',metrics.mean_squared_error(y_test,rfm_pred))
print('RMSE: ',metrics.mean_squared_error(y_test,rfm_pred, squared=False))

MAE:  64986.67194916364
MSE:  11299569066.295725
RMSE:  106299.4311663789


**Decision Tree Regressor**

In [50]:
from sklearn.tree import DecisionTreeRegressor
tree=DecisionTreeRegressor(max_depth=1)
tree.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=1)

In [51]:
tree_pred = tree.predict(X_test)

In [52]:
print('MAE: ',metrics.mean_absolute_error(y_test,tree_pred))
print('MSE: ',metrics.mean_squared_error(y_test,tree_pred))
print('RMSE: ',metrics.mean_squared_error(y_test,tree_pred, squared=False))

MAE:  135223.9116021169
MSE:  34862462676.02295
RMSE:  186714.92354930536


In [53]:
print('R square: ',tree.score(X_train , y_train))

R square:  0.3360290893804655
